# Datasets
In this tutorial, we will dive deeper into the datasets available through `pEYES`. We will explore the datasets available, examine the type of data they contain, and show how to access them both from a local directory and from the internet.

In [1]:
import os
import json

import numpy as np
import pandas as pd

import peyes

_DATASETS_DIR = r'S:\Lab-Shared\Experiments\pEYES\datasets'

## Availble Datasets
The `pEYES` package provides an API to download and parse four datasets: "lund2013", "irf", "hfc" and "gazecom".   
All datasets contains eye-tracking data (t-x-y coordinates), as well as human annotations from at least one annotator. Additionally, the datasets include recording-specific metadata, such as the screen resolution, the sampling rate, viewer distance, and more.    
Thorough descriptions of these datasets is available in Table 1 from article "Evaluating eye movement event detection: A review of the state of the art." (Startsev, M., & Zemblys, R., 2023).

### The `datasets` Submodule
  
The datasets are stored in the `pEYES` package and can be accessed using the `peyes.datasets` submodule.
The module contains an API to download and parse the datasets, or load a parsed version of the dataset from a local directory. This is done by calling the `load_dataset` function with the dataset name as an argument. Additional arguments can be passed to specify a directory from which to load the dataset or save the parsed dataset once it is loaded.  
  
Another API available in the `peyes.datasets` submodule is the `get_metadata` function. This function returns a dictionary containing the metadata of the dataset: article citation, URL, license, etc.  

### Descriptive Statistics
We will extract descriptive statistics for each of the labelers in the dataset: number of labeled samples & trials, distribution of labels, etc.

In [2]:
def _labeler_statistics(dataset: pd.DataFrame, labeler: str) -> dict:
    labeler_data = dataset[dataset[labeler].notnull()]
    labels_distribution = labeler_data[labeler].value_counts(normalize=True).sort_index()
    labels_distribution.index = labels_distribution.index.map(peyes.parse_label)
    labeler_statistics = {
        "num_samples": len(labeler_data),
        "num_subjects": labeler_data[peyes.constants.SUBJECT_ID_STR].nunique(),
        "num_trials": labeler_data[peyes.constants.TRIAL_ID_STR].nunique(),
        "num_labels": labeler_data[labeler].nunique(),
        "labels_distribution": (100 * labels_distribution).to_dict(),
    }
    return labeler_statistics


def _extended_metadata(dataset: pd.DataFrame, metadata: dict) -> dict:
    metadata = metadata.copy()
    metadata["num_samples"] = len(dataset)
    metadata["num_subjects"] = dataset[peyes.constants.SUBJECT_ID_STR].nunique()
    metadata["num_trials"] = dataset[peyes.constants.TRIAL_ID_STR].nunique()
    metadata["sampling_rate"] = peyes._utils.event_utils.calculate_sampling_rate(dataset[peyes.constants.T].values)
    metadata["stimuli"] = dataset[peyes.constants.STIMULUS_TYPE_STR].unique().tolist()
    metadata["raters"] = list(set(dataset.columns) - {
        # Remove non-rater columns
        peyes.constants.TRIAL_ID_STR, peyes.constants.SUBJECT_ID_STR,
        peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR,
        peyes.constants.T, peyes.constants.X, peyes.constants.Y, peyes.constants.PUPIL,
        peyes.constants.PIXEL_SIZE_STR, peyes.constants.VIEWER_DISTANCE_STR,
        "subject_group", "v",
    })
    for rtr in metadata["raters"]:
        metadata[rtr] = _labeler_statistics(dataset, rtr)
    return metadata


def load(
        dataset_name: str, directory: str = _DATASETS_DIR, save: bool = False, verbose: bool = False
) -> (pd.DataFrame, dict):
    """
    Load a dataset from a local directory or download it from the internet.
    The dataset is parsed and returned as a pandas DataFrame.
    The metadata of the dataset is also returned as a dictionary.
    
    :param dataset_name: The name of the dataset to load.
    :param directory: The directory from which to load the dataset or save the parsed dataset.
    :param save: Whether to save the parsed dataset to the directory.
    :param verbose: Whether to print information about the dataset.
    
    :return: The parsed dataset and the metadata of the dataset.
    """
    from peyes.datasets.load_dataset import load_dataset
    dataset = load_dataset(dataset_name, directory, save, verbose)
    metadata = peyes.datasets.get_metadata(dataset_name, False)
    metadata = _extended_metadata(dataset, metadata)
    return dataset, metadata


def pretty_print(metadata: dict, indent: int = 4):
    out = json.dumps(metadata, indent=indent)
    print(out)

#### Lund2013 Dataset

In [46]:
lund_data, lund_metadata = load("lund2013", directory=_DATASETS_DIR, save=False, verbose=True)
lund_stats = pd.concat(
    {rtr: pd.json_normalize(lund_metadata[rtr], sep="_") for rtr in lund_metadata["raters"]},
).droplevel(1)

pretty_print(lund_metadata)
lund_stats

Dataset Lund2013 not found in directory S:\Lab-Shared\Experiments\pEYES\datasets.
Downloading...


Processing Files: 100%|██████████| 69/69 [00:00<00:00, 277.07it/s]


Saved dataset to S:\Lab-Shared\Experiments\pEYES\datasets\Lund2013.pkl
{
    "name": "Lund2013",
    "url": "https://github.com/richardandersson/EyeMovementDetectorEvaluation/archive/refs/heads/master.zip",
    "articles": [
        "Andersson, R., Larsson, L., Holmqvist, K., Stridh, M., & Nystr\u00f6m, M. (2017): One algorithm to rule them all? An evaluation and discussion of ten eye movement event-detection algorithms. Behavior Research Methods, 49(2), 616-637.",
        "Zemblys, R., Niehorster, D. C., Komogortsev, O., & Holmqvist, K. (2018). Using machine learning to detect events in eye-tracking data. Behavior Research Methods, 50(1), 160\u2013181."
    ],
    "license": "GNU GPL-3.0",
    "num_samples": 102227,
    "num_subjects": 20,
    "num_trials": 33,
    "sampling_rate": 500.0,
    "stimuli": [
        "moving_dot",
        "image",
        "video"
    ],
    "raters": [
        "MN",
        "RA"
    ],
    "MN": {
        "num_samples": 102227,
        "num_subjects": 20,

,num_samples,num_subjects,num_trials,num_labels,labels_distribution_0,labels_distribution_1,labels_distribution_2,labels_distribution_3,labels_distribution_4,labels_distribution_5
MN,102227,20,33,6,0.274878,62.919777,7.305311,4.454792,20.858482,4.186761
RA,102220,20,33,6,0.181960,57.963217,7.620818,4.154764,25.765017,4.314224


#### IRF Dataset

In [4]:
irf_data, irf_metadata = load("irf", directory=_DATASETS_DIR, save=False, verbose=True)
irf_stats = pd.concat(
    {rtr: pd.json_normalize(irf_metadata[rtr], sep="_") for rtr in irf_metadata["raters"]},
).droplevel(1)

pretty_print(irf_metadata)
irf_stats

{
    "name": "IRF",
    "url": "https://github.com/r-zemblys/irf/archive/refs/heads/master.zip",
    "articles": [
        "Zemblys, Raimondas and Niehorster, Diederick C and Komogortsev, Oleg and Holmqvist, Kenneth. Using machine learning to detect events in eye-tracking data. Behavior Research Methods, 50(1), 160\u2013181 (2018)."
    ],
    "license": "MIT",
    "num_samples": 486016,
    "num_subjects": 6,
    "num_trials": 6,
    "sampling_rate": 1000000.0000001101,
    "stimuli": [
        "moving_dot"
    ],
    "raters": [
        "RZ"
    ],
    "RZ": {
        "num_samples": 486016,
        "num_subjects": 6,
        "num_trials": 6,
        "num_labels": 5,
        "labels_distribution": {
            "0": 0.33908348696339213,
            "1": 86.76792533579142,
            "2": 5.653517579668159,
            "3": 3.0007242559915723,
            "5": 4.238749341585462
        }
    }
}


,num_samples,num_subjects,num_trials,num_labels,labels_distribution_0,labels_distribution_1,labels_distribution_2,labels_distribution_3,labels_distribution_5
RZ,486016,6,6,5,0.339083,86.767925,5.653518,3.000724,4.238749


#### HFC Dataset

In [5]:
hfc_data, hfc_metadata = load("hfc", directory=_DATASETS_DIR, save=False, verbose=True)
hfc_stats = pd.concat(
    {rtr: pd.json_normalize(hfc_metadata[rtr], sep="_") for rtr in hfc_metadata["raters"]},
).droplevel(1)

pretty_print(hfc_metadata)
hfc_stats

{
    "name": "HFC",
    "url": "https://github.com/dcnieho/humanFixationClassification/archive/refs/heads/master.zip",
    "articles": [
        "Hooge, I.T.C., Niehorster, D.C., Nystr\u00f6m, M., Andersson, R. & Hessels, R.S. (2018). Is human classification by experienced untrained observers a gold standard in fixation detection?",
        "Hessels, R.S., Hooge, I.T.C., & Kemner, C. (2016). An in-depth look at saccadic search in infancy. Journal of Vision, 16(8), 10.",
        "Startsev, M., Zemblys, R. Evaluating Eye Movement Event Detection: A Review of the State of the Art. Behav Res 55, 1653\u20131714 (2023)"
    ],
    "license": "CC NC-BY-SA 4.0",
    "num_samples": 1267692,
    "num_subjects": 70,
    "num_trials": 70,
    "sampling_rate": 300.0300030002925,
    "stimuli": [
        "free_viewing",
        "search_task"
    ],
    "raters": [
        "JV",
        "MS",
        "IH",
        "KH",
        "TC",
        "RH",
        "JB",
        "PZ",
        "MN",
        "R

,num_samples,num_subjects,num_trials,num_labels,labels_distribution_0,labels_distribution_1
JV,1267692,70,70,2,28.952774,71.047226
MS,1267692,70,70,2,27.788453,72.211547
IH,1267692,70,70,2,23.150103,76.849897
KH,1267692,70,70,2,29.248114,70.751886
TC,1267692,70,70,2,29.728988,70.271012
RH,1267692,70,70,2,25.199496,74.800504
JB,1267692,70,70,2,28.538162,71.461838
PZ,1267692,70,70,2,33.378139,66.621861
MN,1267692,70,70,2,27.861342,72.138658
RA,1267692,70,70,2,29.250954,70.749046


#### Gazecom Dataset

In [6]:
gazecom_data, gazecom_metadata = load("gazecom", directory=_DATASETS_DIR, save=False, verbose=True)
gazecom_stats = pd.concat(
    {rtr: pd.json_normalize(gazecom_metadata[rtr], sep="_") for rtr in gazecom_metadata["raters"]},
).droplevel(1)

pretty_print(gazecom_metadata)
gazecom_stats

{
    "name": "GazeCom",
    "url": "https://gin.g-node.org/ioannis.agtzidis/gazecom_annotations/archive/master.zip",
    "articles": [
        "Agtzidis, I., Startsev, M., & Dorr, M. (2016a). In the pursuit of (ground) truth: A hand-labelling tool for eye movements recorded during dynamic scene viewing. In 2016 IEEE second workshop on eye tracking andvisualization (ETVIS) (pp. 65\u201368).",
        "Michael Dorr, Thomas Martinetz, Karl Gegenfurtner, and Erhardt Barth. Variability of eye movements when viewing dynamic natural scenes. Journal of Vision, 10(10):1-17, 2010.",
        "Startsev, M., Agtzidis, I., & Dorr, M. (2016). Smooth pursuit. http://michaeldorr.de/smoothpursuit/",
        "Startsev, M., Zemblys, R. Evaluating Eye Movement Event Detection: A Review of the State of the Art. Behav Res 55, 1653\u20131714 (2023)"
    ],
    "license": "GNU GPL-3.0",
    "num_samples": 12954168,
    "num_subjects": 54,
    "num_trials": 2532,
    "sampling_rate": 250.0,
    "stimuli": [
  

,num_samples,num_subjects,num_trials,num_labels,labels_distribution_0,labels_distribution_1,labels_distribution_2,labels_distribution_4
HL_FINAL,12954168,54,2532,4,5.903490,72.545053,10.532216,11.019241
HL1,12954168,54,2532,4,5.453635,72.248368,10.886658,11.411339
HL2,12954168,54,2532,4,5.291224,77.081932,10.578649,7.048195
